In [1]:
import numpy as np
import serial
import pandas as pd
import os
import tqdm
import time

In [8]:
# Serial object (macOS)
PORT = '/dev/cu.usbmodem14201'
BAUD = 115200
try:
    GATEWAY = serial.Serial(PORT, BAUD)
    print('GateWay opened')
except serial.SerialException:
    print('GateWay not available')

GateWay opened


# Stress Test

In [10]:
TIMEOUT = 0.25
start_time = time.time()
while True:
    # Send 's' to sensor 1 to start streaming data
    GATEWAY.write(b'1')
    #print("Probing Sensor")
    # Wait for data from SENSOR1
    probe_time = time.time()
    while GATEWAY.in_waiting < 1:
        s1_elapsed_time = time.time() - probe_time
        if s1_elapsed_time > TIMEOUT:
            # means the sensor is not responding
            print("Sensor1 not responding... Reprobing")
            # reprobing
            GATEWAY.write(b'1')
            probe_time = time.time()

        pass
    data1 = GATEWAY.readline().decode('utf-8').strip()
    
    #print("Probing Reference")
    # Send 's' to sensor 2 to start streaming data
    GATEWAY.write(b'2')

    probe_time = time.time()
    # Wait for data from SENSOR2
    while GATEWAY.in_waiting < 1:
        s2_elapsed_time = time.time() - probe_time
        if s2_elapsed_time > TIMEOUT:
            # means the sensor is not responding
            print("Sensor2 not responding... Reprobing")
            # reprobing
            GATEWAY.write(b'2')
            probe_time = time.time()
        pass
    data2 = GATEWAY.readline().decode('utf-8').strip()


    #print(data1, data2)
    print("Elapsed time in Seconds: ", time.time() - start_time)
    if time.time() - start_time > 600:
        break

Elapsed time in Seconds:  0.015596866607666016
Elapsed time in Seconds:  0.03053903579711914
Elapsed time in Seconds:  0.0451359748840332
Elapsed time in Seconds:  0.05993294715881348
Elapsed time in Seconds:  0.07455277442932129
Elapsed time in Seconds:  0.09161615371704102
Elapsed time in Seconds:  0.10662484169006348
Elapsed time in Seconds:  0.12135004997253418
Elapsed time in Seconds:  0.13646507263183594
Elapsed time in Seconds:  0.15121006965637207
Elapsed time in Seconds:  0.1658618450164795
Elapsed time in Seconds:  0.18055295944213867
Elapsed time in Seconds:  0.19509482383728027
Elapsed time in Seconds:  0.21087288856506348
Elapsed time in Seconds:  0.2256028652191162
Elapsed time in Seconds:  0.24086403846740723
Elapsed time in Seconds:  0.25595808029174805
Elapsed time in Seconds:  0.27106595039367676
Elapsed time in Seconds:  0.28705501556396484
Elapsed time in Seconds:  0.3057107925415039
Elapsed time in Seconds:  0.3205420970916748
Elapsed time in Seconds:  0.3387379646

KeyboardInterrupt: 

# Baseline capture

In [ ]:
# pands df with gx, gy, gz, ax, ay, az, mx, my, mz, roll, pitch, heading, qw, qx, qy, qz
# Initialize the dataframe with the required columns
df_baseline = pd.DataFrame(columns=['gx1', 'gy1', 'gz1', 'ax1', 'ay1', 'az1', 'mx1', 'my1', 'mz1', 
                           'roll1', 'pitch1', 'heading1', 
                           'gx2', 'gy2', 'gz2', 'ax2', 'ay2', 'az2', 'mx2', 'my2', 'mz2', 
                           'roll2', 'pitch2', 'heading2'])



target_counter = input('Enter number of samples: ')
target_counter = int(target_counter)

for _ in tqdm.tqdm(range(target_counter)):
    # Send 's' to sensor 1 to start streaming data
    GATEWAY.write(b'1')
    
    # Wait for data from SENSOR1
    while GATEWAY.in_waiting < 1:
        pass
    data1 = GATEWAY.readline().decode('utf-8').strip()
    
    # Send 's' to sensor 2 to start streaming data
    GATEWAY.write(b'2')

    # Wait for data from SENSOR2
    while GATEWAY.in_waiting < 1:
        pass
    data2 = GATEWAY.readline().decode('utf-8').strip()
    
    #print(data1)
    #print(data2)

    # Split and convert SENSOR1 data to float
    data1 = [float(i) for i in data1.split(',')]
    # print(data1)
    
    # Split and convert SENSOR2 data to float
    data2 = [float(i) for i in data2.split(',')]
    # print(data2)
    
    
    # Combine both sensor data and append distance
    combined_data = data1 + data2

    # Create a temporary dataframe for the current sample
    temp_df = pd.DataFrame([combined_data], columns=['gx1', 'gy1', 'gz1', 'ax1', 'ay1', 'az1', 'mx1', 'my1', 'mz1', 
                                                     'roll1', 'pitch1', 'heading1', 
                                                     'gx2', 'gy2', 'gz2', 'ax2', 'ay2', 'az2', 'mx2', 'my2', 'mz2', 
                                                     'roll2', 'pitch2', 'heading2'])
    
    # Append the data to the main dataframe using pd.concat
    df_baseline = pd.concat([df_baseline, temp_df], ignore_index=True)

    # Wait for 10ms
    time.sleep(0.01)


# check if data folder exists
if not os.path.exists('data'):
    os.makedirs('data')


# save df to csv
file_name = 'data/data_vr' + '.csv'

# Save the dataframe to a csv file
df_baseline.to_csv(file_name, index=False)


# convert roll, pitch, heading to integers
df_baseline['roll1'] = df_baseline['roll1'].astype(int)
df_baseline['pitch1'] = df_baseline['pitch1'].astype(int)
df_baseline['heading1'] = df_baseline['heading1'].astype(int)
df_baseline['roll2'] = df_baseline['roll2'].astype(int)
df_baseline['pitch2'] = df_baseline['pitch2'].astype(int)
df_baseline['heading2'] = df_baseline['heading2'].astype(int)

# sort the columns based on roll2
df_baseline = df_baseline.sort_values(by=['roll2'])

# make average of the rows with same roll1, pitch1, heading1, roll2, pitch2, heading2
df_baseline = df_baseline.groupby(['roll1', 'pitch1', 'heading1', 'roll2', 'pitch2', 'heading2']).mean().reset_index()


# save df to csv
file_name = 'data/data_baseline' + '.csv'

# Save the dataframe to a csv file
df_baseline.to_csv(file_name, index=False)



# Virtual Alignment Parameter Estimation

In [5]:
# pands df with gx, gy, gz, ax, ay, az, mx, my, mz, roll, pitch, heading, qw, qx, qy, qz
# Initialize the dataframe with the required columns
df = pd.DataFrame(columns=['gx1', 'gy1', 'gz1', 'ax1', 'ay1', 'az1', 'mx1', 'my1', 'mz1', 
                           'roll1', 'pitch1', 'heading1', 
                           'gx2', 'gy2', 'gz2', 'ax2', 'ay2', 'az2', 'mx2', 'my2', 'mz2', 
                           'roll2', 'pitch2', 'heading2', 'dist'])

# Read data from serial port
dist = input('Enter distance: ')
dist = float(dist)

target_counter = input('Enter number of samples: ')
target_counter = int(target_counter)

for _ in tqdm.tqdm(range(target_counter)):
    # Send 's' to sensor 1 to start streaming data
    GATEWAY.write(b'1')
    
    # Wait for data from SENSOR1
    while GATEWAY.in_waiting < 1:
        pass
    data1 = GATEWAY.readline().decode('utf-8').strip()
    
    # Send 's' to sensor 2 to start streaming data
    GATEWAY.write(b'2')

    # Wait for data from SENSOR2
    while GATEWAY.in_waiting < 1:
        pass
    data2 = GATEWAY.readline().decode('utf-8').strip()
    
    #print(data1)
    #print(data2)

    # Split and convert SENSOR1 data to float
    data1 = [float(i) for i in data1.split(',')]
    
    # Split and convert SENSOR2 data to float
    data2 = [float(i) for i in data2.split(',')]
    
    
    # Combine both sensor data and append distance
    combined_data = data1 + data2
    combined_data.append(dist)

    # Create a temporary dataframe for the current sample
    temp_df = pd.DataFrame([combined_data], columns=['gx1', 'gy1', 'gz1', 'ax1', 'ay1', 'az1', 'mx1', 'my1', 'mz1', 
                                                     'roll1', 'pitch1', 'heading1', 
                                                     'gx2', 'gy2', 'gz2', 'ax2', 'ay2', 'az2', 'mx2', 'my2', 'mz2', 
                                                     'roll2', 'pitch2', 'heading2', 'dist'])
    
    # Append the data to the main dataframe using pd.concat
    df = pd.concat([df, temp_df], ignore_index=True)

    # Wait for 10ms
    time.sleep(0.01)


# check if data folder exists
if not os.path.exists('data'):
    os.makedirs('data')

# save df to csv
file_name = 'data/data_' + str(dist) + '.csv'

# check if file exists
for i in range(1000):
    if os.path.exists(file_name):
        file_name = 'data/data_' + str(dist) + '_' + str(i) + '.csv'
    else:
        break

df.to_csv(file_name, index=False)



  0%|          | 0/100 [00:00<?, ?it/s]/var/folders/w2/57pykhpx3v715wp15f4kdzhh0000gp/T/ipykernel_61560/4170598149.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)
100%|██████████| 100/100 [00:04<00:00, 21.14it/s]


In [46]:
# Close the serial port
GATEWAY.close()

In [47]:
df_baseline = pd.read_csv('data/data_baseline.csv')
df_vr = pd.read_csv('data/data_vr.csv')

In [48]:
# Create a function to calculate the Euclidean distance between rows
def calculate_distance(df_baseline, row):
    return np.sqrt(
        (df_baseline['roll1'] - row['roll1']) ** 2 +
        (df_baseline['pitch1'] - row['pitch1']) ** 2 +
        (df_baseline['heading1'] - row['heading1']) ** 2 +
        (df_baseline['roll2'] - row['roll2']) ** 2 +
        (df_baseline['pitch2'] - row['pitch2']) ** 2 +
        (df_baseline['heading2'] - row['heading2']) ** 2
    )

# Optimized function to find the closest row in df_baseline for each row in df_vr and subtract values
def find_and_subtract(df_vr, df_baseline):
    # Iterate over rows in df_vr
    for index, row in df_vr.iterrows():
        # Calculate the distance between the current row and all rows in df_baseline
        distances = calculate_distance(df_baseline, row)
        
        # Find the index of the closest row in df_baseline
        closest_index = distances.idxmin()
        
        # Perform the subtraction on mx1, my1, mz1, mx2, my2, mz2
        df_vr.at[index, 'mx1'] -= df_baseline.at[closest_index, 'mx1']
        df_vr.at[index, 'my1'] -= df_baseline.at[closest_index, 'my1']
        df_vr.at[index, 'mz1'] -= df_baseline.at[closest_index, 'mz1']
        df_vr.at[index, 'mx2'] -= df_baseline.at[closest_index, 'mx2']
        df_vr.at[index, 'my2'] -= df_baseline.at[closest_index, 'my2']
        df_vr.at[index, 'mz2'] -= df_baseline.at[closest_index, 'mz2']

    return df_vr

# Call the function
df_vr = find_and_subtract(df_vr, df_baseline)

In [49]:
from sklearn.linear_model import LinearRegression

# Assuming df is your DataFrame and the dataset is already loaded
X = df_vr[['mx2', 'my2', 'mz2']]  # Independent variables XR, YR, ZR

# Regression for XVR
XVR = df_vr['mx1']
reg_XVR = LinearRegression()
reg_XVR.fit(X, XVR)
ax, bx, cx = reg_XVR.coef_
dx = reg_XVR.intercept_

# Regression for YVR
YVR = df_vr['my1']
reg_YVR = LinearRegression()
reg_YVR.fit(X, YVR)
ay, by, cy = reg_YVR.coef_
dy = reg_YVR.intercept_

# Regression for ZVR
ZVR = df_vr['mz1']
reg_ZVR = LinearRegression()
reg_ZVR.fit(X, ZVR)
az, bz, cz = reg_ZVR.coef_
dz = reg_ZVR.intercept_

# Output the coefficients
print(f"XVR = {ax} * XR + {bx} * YR + {cx} * ZR + {dx}")
print(f"YVR = {ay} * XR + {by} * YR + {cy} * ZR + {dy}")
print(f"ZVR = {az} * XR + {bz} * YR + {cz} * ZR + {dz}")


XVR = -0.04838958374914194 * XR + -0.0293806035280404 * YR + 0.012915074248156884 * ZR + 0.002196403571507596
YVR = 0.003961772264919089 * XR + -0.011191365720206181 * YR + -0.023897067525894065 * ZR + -0.0037991112598711654
ZVR = 0.013618067240529615 * XR + 0.20871153042749133 * YR + 0.17603269331043395 * ZR + -0.03927379520069467


In [36]:
pred_XVR = reg_XVR.predict(X)
XVR_error = XVR - pred_XVR  # Residuals (errors) for XVR
XVR_MSE = np.mean(XVR_error**2)  # Mean Squared Error for XVR
print(f"Mean Squared Error for XVR: {XVR_MSE}")

pred_YVR = reg_YVR.predict(X)
YVR_error = YVR - pred_YVR  # Residuals (errors) for YVR
YVR_MSE = np.mean(YVR_error**2)  # Mean Squared Error for YVR
print(f"Mean Squared Error for YVR: {YVR_MSE}")

pred_ZVR = reg_ZVR.predict(X)
ZVR_error = ZVR - pred_ZVR  # Residuals (errors) for ZVR
ZVR_MSE = np.mean(ZVR_error**2)  # Mean Squared Error for ZVR
print(f"Mean Squared Error for ZVR: {ZVR_MSE}")


Mean Squared Error for XVR: 0.0
Mean Squared Error for YVR: 1.262177448353619e-29
Mean Squared Error for ZVR: 5.048709793414476e-29


# Test VR

In [37]:
# get the average of each column from the dataframe
avg = df_vr.mean( axis=0)

Xr  = avg['mx2']
Yr  = avg['my2']
Zr  = avg['mz2']

# Predict the XVR, YVR, ZVR
XVr = ax * Xr + bx * Yr + cx * Zr + dx
YVr = ay * Xr + by * Yr + cy * Zr + dy
ZVr = az * Xr + bz * Yr + cz * Zr + dz

# Subtract the predicted values from the sensor values
Xx = avg['mx1'] - XVr
Yy = avg['my1'] - YVr
Zz = avg['mz1'] - ZVr

# Calculate RSSI
RSSI = np.sqrt(Xx**2 + Yy**2 + Zz**2)
print(f"RSSI: {RSSI}")

RSSI: 0.0
